In [11]:
import torch
import matplotlib.pyplot as plt

%matplotlib inline

In [55]:
random_seed=12321132

In [12]:
words = open("names.txt", 'r').read().splitlines()

words[:4]

['emma', 'olivia', 'ava', 'isabella']

## MLP

Implementation - 

1. Create training data - Xs and Ys. Depending upon the context length, we need to create a training data.
2. Initialise the weights
3. Forward pass -
   1. Calculate the logits
   2. Calculate the losss  
4. Backward pass -
   1. Calculate the gradients
5. Update the weights

### Create Training data

In [33]:
chrs = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chrs)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [53]:
Xs, Ys = [],[]

block_size = 3 # context length
n_emb = 2      # embedding size for all inputs
n_hidden = 100 # count of neurons in hidden layer.

for w in words[:1]:
    w += '.'
    context = [0] * block_size
    for s in w:
        ix = stoi[s]
        Xs.append(context)
        Ys.append(ix)
        print(f'{"".join(itos[i] for i in context)} ===> {s}')
        context = context[1:] + [ix]

Xs = torch.tensor(Xs)
Ys = torch.tensor(Ys)

... ===> e
..e ===> m
.em ===> m
emm ===> a
mma ===> .


### Initialise the Weights

In [78]:
g = torch.Generator().manual_seed(random_seed)

n_pi = len(stoi)                                                 # count of possible input values
C = torch.randn((n_pi, n_emb), generator = g)                    # embedding matrix
W1 = torch.randn((block_size * n_emb, n_hidden),generator = g)   # hidden layer weights
b1 = torch.randn((n_hidden), generator = g)                      # hidden layer biases
W2 = torch.randn((n_hidden, n_pi), generator = g)                # output layer weights
b2 = torch.randn((n_pi), generator = g)                          # output layer biases

### Forward pass

In [82]:
emb = C[Xs]
h = torch.tanh(emb.view(-1,block_size * n_emb) @ W1 + b1)
logits = h @ W2 + b2
e = torch.exp(logits)
probs = (e/e.sum(1, keepdim=True))
probs[torch.arange(probs.shape[0]), Ys]
loss = F.cross_entropy(logits, Ys)

emb.shape, h.shape, logits.shape, 

(torch.Size([5, 3, 2]), torch.Size([5, 100]), torch.Size([5, 27]))

In [85]:
probs.shape

torch.Size([5, 27])

tensor([2.3052e-07, 1.3760e-05, 1.9607e-06, 4.1155e-10, 1.1659e-02])